In [ ]:
#cd "enfermedades_cardiacas_grupo_1"
# Ejecutar en la terminal

[WinError 3] El sistema no puede encontrar la ruta especificada: '/content/enfermedades_cardiacas_grupo_1'
c:\Users\rosme\OneDrive\Documentos\UNAL\Maestría\Semestre 2\3.MetodologiasAgiles


In [5]:
import os
drive_id = "1VpJLIEIIely_NcvUVKy0F5IXGUkr3e_S" # reemplace aquí el id de su carpeta
os.environ["DRIVEID"] = drive_id

In [ ]:
import os
drive_id = "1VpJLIEIIely_NcvUVKy0F5IXGUkr3e_S" # reemplace aquí el id de su carpeta
os.environ["DRIVEID"] = drive_id

In [6]:
import json
with open("credentials.json") as f:
    os.environ["GDRIVE_CREDENTIALS_DATA"] = f.read()

In [ ]:
dvc remote modify storage gdrive_service_account_json_file_path C:\Users\rosme\credentials.json

In [8]:
os.getcwd()

'c:\\Users\\rosme\\OneDrive\\Documentos\\UNAL\\Maestría\\Semestre 2\\3.MetodologiasAgiles'

## Códigos

In [ ]:
# dvc init

In [ ]:
dvc remote add -d storage gdrive://1VpJLIEIIely_NcvUVKy0F5IXGUkr3e_S


In [ ]:
# Setting 'storage' as a default remote.

In [9]:
dvc remote modify storage gdrive_service_account_json_file_path "c:\\Users\\rosme\\OneDrive\\Documentos\\UNAL\\credentials.json"


SyntaxError: invalid syntax (1162472146.py, line 1)

In [ ]:
# dvc pull

In [1]:
import pandas as pd

In [6]:
df_heart_2022_no_nans = pd.read_csv("heart_2022_no_nans.csv")

In [7]:
df_heart_2022_no_nans

,State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,LastCheckupTime,PhysicalActivities,SleepHours,RemovedTeeth,HadHeartAttack,...,HeightInMeters,WeightInKilograms,BMI,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,TetanusLast10Tdap,HighRiskLastYear,CovidPos
0,Alabama,Female,Very good,4.0,0.0,Within past year (anytime less than 12 months ...,Yes,9.0,None of them,No,...,1.60,71.67,27.99,No,No,Yes,Yes,"Yes, received Tdap",No,No
1,Alabama,Male,Very good,0.0,0.0,Within past year (anytime less than 12 months ...,Yes,6.0,None of them,No,...,1.78,95.25,30.13,No,No,Yes,Yes,"Yes, received tetanus shot but not sure what type",No,No
2,Alabama,Male,Very good,0.0,0.0,Within past year (anytime less than 12 months ...,No,8.0,"6 or more, but not all",No,...,1.85,108.86,31.66,Yes,No,No,Yes,"No, did not receive any tetanus shot in the pa...",No,Yes
3,Alabama,Female,Fair,5.0,0.0,Within past year (anytime less than 12 months ...,Yes,9.0,None of them,No,...,1.70,90.72,31.32,No,No,Yes,Yes,"No, did not receive any tetanus shot in the pa...",No,Yes
4,Alabama,Female,Good,3.0,15.0,Within past year (anytime less than 12 months ...,Yes,5.0,1 to 5,No,...,1.55,79.38,33.07,No,No,Yes,Yes,"No, did not receive any tetanus shot in the pa...",No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246017,Virgin Islands,Male,Very good,0.0,0.0,Within past 2 years (1 year but less than 2 ye...,Yes,6.0,None of them,No,...,1.78,102.06,32.28,Yes,No,No,No,"Yes, received tetanus shot but not sure what type",No,No
246018,Virgin Islands,Female,Fair,0.0,7.0,Within past year (anytime less than 12 months ...,Yes,7.0,None of them,No,...,1.93,90.72,24.34,No,No,No,No,"No, did not receive any tetanus shot in the pa...",No,Yes
246019,Virgin Islands,Male,Good,0.0,15.0,Within past year (anytime less than 12 months ...,Yes,7.0,1 to 5,No,...,1.68,83.91,29.86,Yes,Yes,Yes,Yes,"Yes, received tetanus shot but not sure what type",No,Yes
246020,Virgin Islands,Female,Excellent,2.0,2.0,Within past year (anytime less than 12 months ...,Yes,7.0,None of them,No,...,1.70,83.01,28.66,No,Yes,Yes,No,"Yes, received tetanus shot but not sure what type",No,No


## Entrenamiento

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [12]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
#from imblearn.over_sampling import RandomOverSampler
#from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [13]:
X = df_heart_2022_no_nans.drop('HadHeartAttack', axis=1)
#X = pd.get_dummies(X)
y = df_heart_2022_no_nans['HadHeartAttack']

In [14]:
categoricas = X.select_dtypes(include='object').columns.tolist()

# Obtener los nombres de las variables numéricas
numericas = X.select_dtypes(exclude='object').columns.tolist()

In [15]:
tf = ColumnTransformer([
        ('onehot', OneHotEncoder(), categoricas),
        ('scaler', StandardScaler(), numericas)])
X = tf.fit_transform(X)

## Partición de los datos

In [16]:
## Partición de los datos originales
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

In [17]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from joblib import dump, load

In [18]:
from sklearn.linear_model import LogisticRegression
# Crear un pipeline con Regresión Logística
pipeline = Pipeline([
    ('scaler', StandardScaler(with_mean=False)),  # Escalar las características
    ('lr', LogisticRegression(random_state=42))  # Clasificador Regresión Logística
])

# Definir la grilla de hiperparámetros específicos de Regresión Logística
param_grid = {
    'lr__C': [0.1, 1, 10],  # Inverso de la fuerza de regularización
    'lr__penalty': ['l1', 'l2']  # Tipo de regularización (L1 o L2)
}

grid_search_LogisticRegression = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, verbose=10)
grid_search_LogisticRegression.fit(X_train, y_train)

# Mostrar los mejores hiperparámetros encontrados
print("Mejores hiperparámetros:")
print(grid_search_LogisticRegression.best_params_)

# Predecir en el conjunto de prueba
y_pred = grid_search_LogisticRegression.predict(X_test)

# Mostrar métricas de evaluación
print("Informe de clasificación:")
Ligistic_regression_report=classification_report(y_test, y_pred)
print(Ligistic_regression_report)


Fitting 5 folds for each of 6 candidates, totalling 30 fits


c:\Users\rosme\miniconda3\lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
15 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\rosme\miniconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\rosme\miniconda3\lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\rosme\miniconda3\lib\site-packages\sklearn\pipeline.py", line 473, in fit
    self._final_estimator.fit(Xt, y, **last_step_params["fit"])
  File "c:\

Mejores hiperparámetros:
{'lr__C': 10, 'lr__penalty': 'l2'}
Informe de clasificación:
              precision    recall  f1-score   support

          No       0.96      0.99      0.97     46513
         Yes       0.58      0.25      0.35      2692

    accuracy                           0.95     49205
   macro avg       0.77      0.62      0.66     49205
weighted avg       0.94      0.95      0.94     49205



In [19]:
# Crear un pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler(with_mean=False)),  # Escalar las características
    ('rf', RandomForestClassifier(random_state=42))  # Clasificador Random Forest
])

# Definir la grilla de hiperparámetros
param_grid = {
    'rf__n_estimators': [10, 20],  # Número de árboles en el bosque
    'rf__max_depth': [2, 5, 10],  # Profundidad máxima de los árboles
    'rf__min_samples_split': [2, 5],  # Número mínimo de muestras requeridas para dividir un nodo
    'rf__min_samples_leaf': [1, 2]  # Número mínimo de muestras requeridas en cada nodo hoja
}

# Crear un objeto GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, verbose=10)

# Entrenar el modelo utilizando GridSearchCV
grid_search.fit(X_train, y_train)

# Mostrar los mejores hiperparámetros encontrados
print("Mejores hiperparámetros:")
print(grid_search.best_params_)

# Predecir en el conjunto de prueba
y_pred = grid_search.predict(X_test)

# Mostrar métricas de evaluación
print("Informe de clasificación:")
Random_forest_report = classification_report(y_test, y_pred)
print(Random_forest_report)


Fitting 5 folds for each of 24 candidates, totalling 120 fits


KeyboardInterrupt: 